<a href="https://colab.research.google.com/github/nonoumasy/LA-County-Covid-Cases/blob/master/LA_County_Covid_Cases_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gspread oauth2client df2gspread
!pip install mapboxgl

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for df2gspread: filename=df2gspread-1.0.4-cp36-none-any.whl size=11953 sha256=bd719be1e9f37c336e35aa3503225b6ea253a6ee4a89a2ae5ddc9f6e23c90805
  Stored in directory: /root/.cache/pip/wheels/19/d2/15/07b0a45a18edadbf0bb3a5655e997925e2b9cbf08b4a95342e
Successfully built df2gspread
ERROR: firebase-admin 4.1.0 has requirement google-api-python-client>=1.7.8, but you'll have google-api-python-client 1.6.7 which is incompatible.
  Found existing installation: google-api-python-client 1.7.12
    Uninstalling google-api-python-client-1.7.12:
      Successfully uninstalled google-api-python-client-1.7.12


     |████████████████████████████████| 51kB 1.7MB/s 
  Created wheel for chroma-py: filename=chroma_py-0.1.0.dev1-cp36-none-any.whl size=5107 sha256=4e4202ad1b7ee49df67b59894169eeaa39e49cc13128e8a28790731fa417e987
  Stored in directory: /root/.cache/pip/wheels/43/3b/8c/3f6d7536b8bef26b7c3be5989f8103513eb949e50a4f9f81cf
Successfully built chroma-py


In [ ]:
import pandas as pd
import numpy as np
#from geopy.geocoders import Nominatim

import gspread 
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

from mapboxgl.utils import create_color_stops, df_to_geojson
from mapboxgl.viz import CircleViz

In [ ]:
# load data from url
df = pd.DataFrame(pd.read_html('http://publichealth.lacounty.gov/media/Coronavirus/locations.htm')[0])

In [ ]:
# skip rows
la_df = df.iloc[50:, :].reset_index(drop=True)

# long beach and pasadena
lb_df = df.iloc[6:8, :]
lb_df.columns = ['locations', 'total_cases', 'case_rate', 'deaths', 'deaths_rate']

In [ ]:
def cleanup_df(df):
  df.columns = df.columns.str.lower().str.replace(' ', '_')
  df.columns = ['locations', 'total_cases', 'case_rate', 'deaths', 'deaths_rate']

  # cleanup values
  df.locations = df.locations.str.replace('*', '').str.replace('City of ', '').str.replace('Los Angeles - ', '').str.replace('Unincorporated - ', '').str.replace('-', '').str.strip()
  #df['total_cases'] = df.total_cases.str.replace('--', '0')
  #df['rate'] = df.rate.str.replace('--', '0')


  # drop some rows
  df = df[df.locations != 'Los Angeles']
  df = df[df.locations != 'Under Investigation']

  #
  df.dropna(subset=['total_cases'], inplace=True)
  df.dropna(subset=['case_rate'], inplace=True)
  df.dropna(subset=['deaths'], inplace=True)
  df.dropna(subset=['deaths_rate'], inplace=True)
  df.drop_duplicates('locations', inplace=True)

  # 
  df['total_cases'] = df['total_cases'].astype('int')
  df['case_rate'] = df['case_rate'].astype('float')
  df['deaths'] = df['deaths'].astype('int')
  df['deaths_rate'] = df['deaths_rate'].astype('float')

  #
  df = df.sort_values('locations')

  # reset index
  df.reset_index(drop=True, inplace=True)

  return df

In [ ]:
la_df = cleanup_df(la_df)

In [ ]:
lb_df.columns = lb_df.columns.str.lower().str.replace(' ', '_')
lb_df.columns = ['locations', 'total_cases', 'case_rate', 'deaths', 'deaths_rate']
lb_df.locations = lb_df.locations.str.replace('-','').str.strip()

lb_df['total_cases'] = lb_df['total_cases'].astype('int')

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [ ]:
# remember to mount Google drive
json_path = '/content/drive/My Drive/jupyter-275007-0fd4fe387056.json'

# Connect to our service account and open Google Sheet
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
gc = gspread.authorize(credentials)

spreadsheet_key = '1S7kIiYZlNIROuj00H_qOv1posb8kec_yjk2aIz4QHeM' 
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet('locations') 
table = worksheet.get_all_values()

# Convert table data into a dataframe
locations_df = pd.DataFrame(table[1:], columns=table[0])

In [ ]:
# merge la and lbf
new_df = pd.concat([la_df, lb_df], ignore_index=True)

#new_df.drop_duplicates('locations', keep='last', inplace=True)
new_df.reset_index(drop=True, inplace=True)

# merge with geocoded locations
temp_df = new_df.merge(locations_df)

# change lat, lon to float
temp_df.lat = temp_df.lat.astype('float')
temp_df.lon = temp_df.lon.astype('float')

In [ ]:
# save to Google Sheets

spreadsheet_key = '1S7kIiYZlNIROuj00H_qOv1posb8kec_yjk2aIz4QHeM' 

d2g.upload(temp_df, spreadsheet_key, 'la_county_coronavirus', credentials=credentials, row_names=False)
#d2g.upload(deaths_df, spreadsheet_key, 'case_deaths', credentials=credentials, row_names=False)

<Worksheet 'la_county_coronavirus' id:991558947>

In [ ]:
# Must be a public token, starting with `pk`
token = 'pk.eyJ1Ijoibm9ub3VtYXN5IiwiYSI6ImNrMTBmY3MycTA1YTEzY3F3ZHZ3eHNsdTAifQ.7r-ppKeBALXFid9Vmpa9Pw'

# Create a geojson file export from a Pandas dataframe
data = df_to_geojson(temp_df, 
              properties=['locations', 'total_cases', 'case_rate', 'deaths', 'deaths_rate'],
              lat='lat', 
              lon='lon', 
              precision=3
              )

# Generate data breaks and color stops from colorBrewer
color_breaks = [0,50,100,250,500,1000,1500]
color_stops = create_color_stops(color_breaks, colors='YlGnBu')

viz = CircleViz(data, 
                height=800,
                radius=4,
                access_token=token,
                color_property = "total_cases",
                stroke_color='#191925',
                stroke_width=1,
                #label_property='total_cases',
                color_stops = color_stops,
                legend_key_shape='circle',
                center = (-118.2437, 34.0522),
                zoom = 9,
                #min_zoom = 10,
                #below_layer = 'waterway-label',
                style='mapbox://styles/nonoumasy/cka8p77pa1iiq1it9mqdqhumu',
              )

viz.label_property = 'total_cases'
viz.label_color = '#191925'
viz.label_size = 10
viz.label_halo_width = 2
viz.label_halo_color = '#ffffff'
viz.create_html('index.html')
viz.show()


In [14]:
with open('index.html', 'w') as f:
    f.write(viz.create_html())

# Geocode

In [ ]:
# prep location for geocoding. Explicitly add ', Los Angeles, California' so the geocoder finds the location

def geocode(df):
  df['loc_prep'] = 'temp'

  for i in range(len(df)):
    df['loc_prep'][i] = str(df.ocations[i]) + ', Los Angeles, California'

  # create and initialize coords column
  df['coords'] = 0.0

  # geocode locations
  geolocator = Nominatim(user_agent="sdfasdf")

  # to space out the geocoding
  from geopy.extra.rate_limiter import RateLimiter

  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  df['coords'] = df.loc_prep.apply(lambda x : geocode(x))

  #
  df['lat'] = df.coords.apply(lambda x: x.latitude if x !=None else 0.0)
  df['lon'] = df.coords.apply(lambda x: x.longitude if x !=None else 0.0)

  return df